In [1]:
import time

import cv2
import matplotlib.pyplot as plt
import numpy as np

import cflib.crtp
from cflib.crazyflie import Crazyflie
from cflib.crazyflie.log import LogConfig
from cflib.crazyflie.syncCrazyflie import SyncCrazyflie
from cflib.crazyflie.syncLogger import SyncLogger
from cflib.positioning.position_hl_commander import PositionHlCommander
from cflib.positioning.motion_commander import MotionCommander

## constants

In [2]:
group_number = 4
uri = f'radio://0/{group_number}/2M'
camera = 2
log_period_ms = 50

## testing camera

In [10]:
cap = cv2.VideoCapture(camera)

while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    
#     hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
#     print(np.mean(hsv[210:270, 290:350, :], axis=(0, 1)))

    mask, largest_contour, area = detect_obstacles()
    print(area)

    # Compute
    cv2.imshow('frame', mask)
#     cv2.imshow('frame', frame)

    # Hit q to quit.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture
cap.release()
cv2.destroyAllWindows()
del cap

0
247.5
232.5
173.0
487.5
179.5


QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to target thread (0x5555b8d8aae0)

QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to target thread (0x5555b8d8aae0)

QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to target thread (0x5555b8d8aae0)

QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to target thread (0x5555b8d8aae0)

QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to target thread (0x5555b8d8aae0)

QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to target thread (0x5555b8d8aae0)

QObject::moveToThread: Current thread (0x5555b8d8aae0) is not the object's thread (0x5555b9644680).
Cannot move to tar

311.5
439.5
333.0
405.5
98.5
128.0
307.5
278.5
255.5
172.0
245.5
199.5
559.0
177.0
146.0
237.5
172.0
234.0
305.5
198.0
328.5
245.0
166.0
351.0
319.0
187.5
220.5
333.0
319.0
222.0
270.0
215.0
186.0
289.0
224.0
207.0
515.5
401.0
217.5
355.0
757.0
257.5
282.5
245.0
131.5
382.0
322.0
179.0
269.0
185.0
208.0
152.5
283.5
112.0
141.0
201.5
353.5
248.5
193.5
106.0
114.0
160.5
229.5
185.0
115.5
187.5
175.5
79.5
83.5
86.5
61.0
36.0
170.0
185.5
142.0
213.5
131.0


## helper functions

In [7]:
def wait_for_position_estimator(scf):
    print('Waiting for estimator to find position...')

    log_config = LogConfig(name='Kalman Variance', period_in_ms=500)
    log_config.add_variable('kalman.varPX', 'float')
    log_config.add_variable('kalman.varPY', 'float')
    log_config.add_variable('kalman.varPZ', 'float')

    var_y_history = [1000] * 10
    var_x_history = [1000] * 10
    var_z_history = [1000] * 10

    threshold = 0.001
    with SyncLogger(scf, log_config) as logger:
        for log_entry in logger:
            data = log_entry[1]

            var_x_history.append(data['kalman.varPX'])
            var_x_history.pop(0)
            var_y_history.append(data['kalman.varPY'])
            var_y_history.pop(0)
            var_z_history.append(data['kalman.varPZ'])
            var_z_history.pop(0)

            min_x = min(var_x_history)
            max_x = max(var_x_history)
            min_y = min(var_y_history)
            max_y = max(var_y_history)
            min_z = min(var_z_history)
            max_z = max(var_z_history)

            print("{} {} {}".
                format(max_x - min_x, max_y - min_y, max_z - min_z))

            if (max_x - min_x) < threshold and (
                    max_y - min_y) < threshold and (
                    max_z - min_z) < threshold:
                break

def set_PID_controller(cf):
    # Set the PID Controller:
    print('Initializing PID Controller')
    cf.param.set_value('stabilizer.controller', '1')
    cf.param.set_value('kalman.resetEstimation', '1')
    time.sleep(0.1)
    cf.param.set_value('kalman.resetEstimation', '0')
    
    wait_for_position_estimator(cf)
    time.sleep(0.1)
        
def ascend_and_hover(cf, height):
    print('ascending')
            
    # Ascend:
    for y in range(15):
        height_multi = -(y / 15 - 1) ** 2 + 1
        cf.commander.send_hover_setpoint(0, 0, 0, height * height_multi)
        time.sleep(0.1)

    # Hover at 1 meter:
    for _ in range(20):
        cf.commander.send_hover_setpoint(0, 0, 0, height)
        time.sleep(0.1)
    
def hover_and_descend(cf, height):
    print('descending')
    
    # Hover at 1 meter:
    for _ in range(30):
        cf.commander.send_hover_setpoint(0, 0, 0, height)
        time.sleep(0.1)
    
    # Descend:
    for y in range(15):
        cf.commander.send_hover_setpoint(0, 0, 0, height * (1 - y / 15))
        time.sleep(0.1)
    
    # Stop all motion:
    for i in range(10):
        cf.commander.send_stop_setpoint()
        time.sleep(0.1)
        
def position_callback(timestamp, data, log_conf):
    global x_cur, y_cur, z_cur
    x_cur = data['kalman.stateX']
    y_cur = data['kalman.stateY']
    z_cur = data['kalman.stateZ']
    
# Sort through contours in the image
def findGreatestContour(contours):
    largest_area = 0
    largest_contour_index = -1
    i = 0
    total_contours = len(contours)

    while i < total_contours:
        area = cv2.contourArea(contours[i])
        if area > largest_area:
            largest_area = area
            largest_contour_index = i
        i += 1

    #print(largest_area)

    return largest_area, largest_contour_index

def detect_obstacles():
    lb1 = (145, 35, 75)
    ub1 = (180, 255, 255)
    lb2 = (0, 75, 75)
    ub2 = (30, 255, 255)

    # Perform contour detection on the input frame.
    hsv1 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of red obstacles in either color range.
    mask1 = cv2.inRange(hsv1, lb1, ub1)
    mask2 = cv2.inRange(hsv2, lb2, ub2)
    # Combine the masks.
    mask = cv2.bitwise_or(mask1, mask2)

#     # Compute
#     cv2.imshow('mask', mask)    

#     # Hit q to quit.
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    area, idx = findGreatestContour(contours)    
    largest_contour = contours[idx] if idx != -1 else None
    
    return mask, largest_contour, area

def detect_book():
    lower_blue = np.array([75,40,170])
    upper_blue = np.array([95,255,255])

    # Perform contour detection on the input frame.
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Compute mask of blue book in either color range.
    mask = cv2.inRange(hsv, lower_blue, upper_blue)

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)
    area, idx = findGreatestContour(contours)
    largest_contour = contours[idx] if idx != -1 else None

    return mask, largest_contour, area

# loop

In [19]:
x_cur = 0.0
y_cur = 0.0
z_cur = 0.0

total_width = 1.6
left_boundary = total_width / 2
right_boundary = -total_width / 2

book_left_boundary = 0.7
book_right_boundary = -book_left_boundary

length = 96 * 0.0254
book_boundary = length
max_boundary = book_boundary + 0.15

obstacle_width = 3 * 0.0254 * 1.5

book_deviation = 0.2
obstacle_deviation = 0.08

min_obstacle_area = 5000
min_book_area = 9000

side_step = 0.06
book_side_step = 0.07
forward_step = 0.12
height = 0.9

# Initialize all the CrazyFlie drivers:
cflib.crtp.init_drivers(enable_debug_driver=False)

# Scan for Crazyflies in range of the antenna:
print('Scanning interfaces for Crazyflies...')
available = cflib.crtp.scan_interfaces()

# List local CrazyFlie devices:
print('Crazyflies found:')
for i in available:
    print(i[0])

if len(available) == 0:
    print('FAILED! FAILED! FAILED!')
else:
    with SyncCrazyflie(uri, cf=Crazyflie(rw_cache='./cache')) as scf:
        cf = scf.cf
        
        # set up logging
        log_conf = LogConfig(name='Position', period_in_ms=log_period_ms)
        log_conf.add_variable('kalman.stateX', 'float')
        log_conf.add_variable('kalman.stateY', 'float')
        log_conf.add_variable('kalman.stateZ', 'float')

        scf.cf.log.add_config(log_conf)
        log_conf.data_received_cb.add_callback(position_callback)
        log_conf.start()
        
        # Initialize and ascend:
        t = time.time()
        elapsed = time.time() - t
        ascended_bool = 0

        # capture the video
        cap = cv2.VideoCapture(camera)
        
        # get the video frames' width and height
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))

        # flag indicating whether to exit the main loop and then descend
        exit_loop = False
        
        # start to hover
        set_PID_controller(cf)
        ascend_and_hover(cf, height)
        time.sleep(1)
        
        while cap.isOpened() and not exit_loop:
            ok, frame = cap.read()
            
            if not ok:
                print('no frame!')
                break
            
            print()
            mask, largest_contour, area = detect_obstacles()
            
            if x_cur > book_boundary:
                print(f"looking for book; x_cur = {x_cur:2f}; book_boundary = {book_boundary:.2f}")
                # look for book here
                mask, largest_contour, area = detect_book()
                
                if area < min_book_area or y_cur < book_right_boundary or y_cur > book_left_boundary:
                    if y_cur < 0:
                        print(f"no book found; area = {area}; going left to {y_cur + book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + book_side_step, height, 0)
                        time.sleep(0.1)
                    else:
                        print(f"no book found; area = {area}; going right to {y_cur - book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - book_side_step, height, 0)
                        time.sleep(0.1)
                    continue
                
                left_edge = np.amin(largest_contour[:, :, 0])
                right_edge = np.amax(largest_contour[:, :, 0])
                book_center = (left_edge + right_edge) / 2
                l_bound = mask.shape[1] * (0.5 - book_deviation)
                r_bound = mask.shape[1] * (0.5 + book_deviation)
                
                if book_center < l_bound:
                    # go right
                    print(f"book found area = {area}; book center = {book_center:.2f}; going left to {y_cur + book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                    cf.commander.send_position_setpoint(x_cur, y_cur + book_side_step, height, 0)
                    time.sleep(0.1)
                elif book_center > r_bound:
                    # go left
                    print(f"book found area = {area}; book center = {book_center:.2f}; going right to {y_cur - book_side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                    cf.commander.send_position_setpoint(x_cur, y_cur - book_side_step, height, 0)
                    time.sleep(0.1)
                else:
                    # land
                    if area < 4000 and x_cur < max_boundary:
                        print(f"book found area = {area}; book center = {book_center:.2f}; going forward to {x_cur + forward_step:2f}")
                        cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, height, 0)
                        time.sleep(0.1)
                    else:
                        print(f"book found area = {area}; book center = {book_center:.2f}; landing")
                        break
                
            elif area > min_obstacle_area:
                # adjust position
                left_edge = np.amin(largest_contour[:,:,0])
                right_edge = np.amax(largest_contour[:,:,0])
                
                
                l_bound = mask.shape[1] * (0.5 - obstacle_deviation)
                c_bound = mask.shape[1] * 0.5
                r_bound = mask.shape[1] * (0.5 + obstacle_deviation)
                
                print(f'looking for obstacle; area = {area}; width: {mask.shape[1]}, left edge: {left_edge}, right edge: {right_edge}')
                
                if right_edge < l_bound or left_edge > r_bound:
                    # go forward
                    print(f"contour not in bound; going forward to {x_cur + forward_step}, y_cur = {y_cur}")
                    cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, height, 0)
                    time.sleep(0.1)
                else:
                    mass_l = np.sum(mask[:, int(l_bound):int(c_bound)])
                    mass_r = np.sum(mask[:, int(c_bound):int(r_bound)])
                    print(f"contour in bound; left mass = {mass_l}, right mass = {mass_r}")
                    
                    if y_cur > left_boundary:
                        print(f"on the boundary, going right to {y_cur - obstacle_width:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - obstacle_width, height, 0)
                        time.sleep(0.1)
                    elif y_cur < right_boundary:
                        print(f"on the boundary, going left to {y_cur + obstacle_width:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + obstacle_width, height, 0)
                        time.sleep(0.1)
                    elif mass_l < mass_r:
                        # go left
                        print(f"going left to {y_cur + side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur + side_step, height, 0)
                        time.sleep(0.1)
                    else:
                        # go right
                        print(f"going right to {y_cur - side_step:.2f}, x_cur = {x_cur:.2f}, y_cur = {y_cur:.2f}")
                        cf.commander.send_position_setpoint(x_cur, y_cur - side_step, height, 0)
                        time.sleep(0.1)
            else:
                # go forward
                print(f"small area = {area}; going forward to {x_cur + forward_step}, y_cur = {y_cur}")
                cf.commander.send_position_setpoint(x_cur + forward_step, y_cur, height, 0)
                time.sleep(0.1)
            
        cap.release()
        
        # descend
        hover_and_descend(cf, height)

cv2.destroyAllWindows()

Scanning interfaces for Crazyflies...


Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'
Exception while scanning for Crazyflie USB: 'NoneType' object has no attribute 'bcdDevice'


Crazyflies found:
radio://0/3/2M
radio://0/4/2M
radio://0/17/2M
radio://0/18/2M
radio://0/3/2M
radio://0/4/2M
radio://0/17/2M
radio://0/18/2M
radio://0/3/2M
radio://0/4/2M
radio://0/17/2M
radio://0/18/2M
radio://0/3/2M
radio://0/4/2M
radio://0/17/2M
radio://0/18/2M
radio://0/3/2M
radio://0/4/2M
radio://0/17/2M
radio://0/18/2M


Error no LogEntry to handle id=1


Initializing PID Controller
Waiting for estimator to find position...
999.9999903096841 999.9999903209728 999.9997914534761
999.9999903096841 999.9999903209728 999.9997914534761
999.9999903096841 999.9999903209728 999.9997942221089
999.9999904292254 999.9999904586548 999.9997942221089
999.9999906745452 999.9999905136638 999.9997942221089
999.9999906745452 999.9999905136638 999.9997942221089
999.9999906745452 999.9999905136638 999.9997942221089
999.9999906745452 999.9999905136638 999.9997942221089
999.9999906745452 999.9999905136638 999.9997942221089
2.0455472622415982e-06 1.8739183360594325e-06 2.5192464818246663e-05
ascending

small area = 0; going forward to 0.2247269031405449, y_cur = -0.11177992820739746

looking for obstacle; area = 14467.5; width: 640, left edge: 290, right edge: 367
contour in bound; left mass = 3423375, right mass = 2015010
going right to -0.19, x_cur = 0.11, y_cur = -0.13

looking for obstacle; area = 12647.5; width: 640, left edge: 241, right edge: 289
contou


looking for obstacle; area = 46858.0; width: 640, left edge: 72, right edge: 237
contour not in bound; going forward to 0.5126420509815216, y_cur = -0.2596583366394043

looking for obstacle; area = 52885.5; width: 640, left edge: 32, right edge: 213
contour not in bound; going forward to 0.5275918793678284, y_cur = -0.2429303675889969

looking for obstacle; area = 52573.0; width: 640, left edge: 28, right edge: 211
contour not in bound; going forward to 0.5466000092029571, y_cur = -0.23863035440444946

looking for obstacle; area = 52798.0; width: 640, left edge: 28, right edge: 233
contour not in bound; going forward to 0.5688371014595032, y_cur = -0.2356129288673401

looking for obstacle; area = 55674.0; width: 640, left edge: 32, right edge: 243
contour not in bound; going forward to 0.5915130031108856, y_cur = -0.23870378732681274

looking for obstacle; area = 52756.5; width: 640, left edge: 61, right edge: 245
contour not in bound; going forward to 0.6263454508781433, y_cur = -0.2


looking for obstacle; area = 29327.0; width: 640, left edge: 144, right edge: 248
contour not in bound; going forward to 1.4728002500534059, y_cur = -0.208474799990654

looking for obstacle; area = 31764.0; width: 640, left edge: 132, right edge: 243
contour not in bound; going forward to 1.4989515495300294, y_cur = -0.21030892431735992

looking for obstacle; area = 33217.5; width: 640, left edge: 121, right edge: 236
contour not in bound; going forward to 1.5281130027770997, y_cur = -0.20377330482006073

looking for obstacle; area = 36671.5; width: 640, left edge: 117, right edge: 229
contour not in bound; going forward to 1.5688551616668702, y_cur = -0.1931580752134323

looking for obstacle; area = 38679.0; width: 640, left edge: 110, right edge: 230
contour not in bound; going forward to 1.5906145524978639, y_cur = -0.1892523318529129

looking for obstacle; area = 42881.5; width: 640, left edge: 97, right edge: 226
contour not in bound; going forward to 1.6091421794891358, y_cur = 


looking for obstacle; area = 41781.5; width: 640, left edge: 509, right edge: 639
contour not in bound; going forward to 2.2704168510437013, y_cur = 0.005222820211201906

looking for obstacle; area = 31792.0; width: 640, left edge: 524, right edge: 639
contour not in bound; going forward to 2.3044463348388673, y_cur = -0.0002456055663060397

looking for obstacle; area = 20057.0; width: 640, left edge: 546, right edge: 639
contour not in bound; going forward to 2.3215581130981446, y_cur = 0.0026013259775936604

looking for obstacle; area = 7058.0; width: 640, left edge: 600, right edge: 639
contour not in bound; going forward to 2.3300989818573, y_cur = 0.003967994824051857

small area = 0; going forward to 2.3423525047302247, y_cur = 0.009376670233905315

small area = 912.0; going forward to 2.36727201461792, y_cur = 0.010998194105923176

small area = 830.5; going forward to 2.3847333145141603, y_cur = 0.011457005515694618

small area = 1315.5; going forward to 2.409698123931885, y_cu

In [18]:
cap.release()